# Group Historical Weather data by region and department

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import xarray as xr
from energy_forecast import ROOT_DIR
from energy_forecast.geography import get_mask_departements, get_mask_regions
from energy_forecast.meteo import calculate_mean_group_value, instant_flux_from_cumul
from energy_forecast.constants import departement_names, region_names, france_bounds
import pandas as pd


/home/antoine/.local/share/hatch/env/virtual/energy-forecast/Jk97fpOc/energy-forecast/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import dask.distributed
dask_client = dask.distributed.Client()

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.diskutils:Found stale lock file and directory '/tmp/dask-scratch-space/worker-3he7lve9', purging
INFO:distributed.diskutils:Found stale lock file and directory '/tmp/dask-scratch-space/worker-x6b8qj3y', purging
INFO:distributed.diskutils:Found stale lock file and directory '/tmp/dask-scratch-space/scheduler-kssq60ro', purging
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:45011
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37723'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33209'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:40105', name: 0, status: init, memory: 0, proce

# Departements

In [5]:
mask_deps = get_mask_departements()
mask_deps

<xarray.DataArray (longitude: 143, latitude: 97)> Size: 111kB
[13871 values with dtype=float64]
Coordinates:
  * longitude  (longitude) float64 1kB -4.7 -4.6 -4.5 -4.4 ... 9.2 9.3 9.4 9.5
  * latitude   (latitude) float64 776B 51.0 50.9 50.8 50.7 ... 41.6 41.5 41.4

## Temperature

In [6]:
filename_all_temperatures = ROOT_DIR / 'data' / 'silver' / 'temperature.nc'

temperature_da = xr.open_dataarray(filename_all_temperatures,
                                   chunks={'time':70}
)
temperature_da

<xarray.DataArray 'temperature' (time: 772, step: 97, latitude: 97,
                                 longitude: 143)> Size: 4GB
dask.array<open_dataset-temperature, shape=(772, 97, 97, 143), dtype=float32, chunksize=(70, 97, 97, 143), chunktype=numpy.ndarray>
Coordinates:
  * step        (step) timedelta64[ns] 776B 00:00:00 ... 4 days 00:00:00
  * latitude    (latitude) float64 776B 51.0 50.9 50.8 50.7 ... 41.6 41.5 41.4
  * longitude   (longitude) float64 1kB -4.7 -4.6 -4.5 -4.4 ... 9.2 9.3 9.4 9.5
  * time        (time) datetime64[ns] 6kB 2022-02-01 2022-02-02 ... 2024-04-08
    valid_time  (time, step) datetime64[ns] 599kB dask.array<chunksize=(70, 97), meta=np.ndarray>
Attributes: (12/29)
    GRIB_paramId:                             167
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      386061
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                2 metre temperature
    GRIB_shortName:                           2t
    GRIB_units:                               K
    long_name:                                2 metre temperature
    units:                                    K
    standard_name:                            air_temperature

In [7]:
mean_temperature_deps = calculate_mean_group_value(
    masks=mask_deps,
    names=departement_names,
    label='departement',
    da_value=temperature_da,
    min_lon=france_bounds['min_lon'],
    max_lon=france_bounds['max_lon'],
    min_lat=france_bounds['min_lat'],
    max_lat=france_bounds['max_lat'],
)
mean_temperature_deps

temperature
time       departement        valid_time             
2022-02-01 Aisne              2022-02-01   273.894684
           Aube               2022-02-01   274.222931
           Calvados           2022-02-01   277.285767
           Cantal             2022-02-01   272.249756
           Eure-et-Loir       2022-02-01   274.448944
...                                               ...
2024-04-08 Val-d'Oise         2024-04-12   283.421600
           Loir-et-Cher       2024-04-12   283.282043
           Mayenne            2024-04-12   283.432007
           Meurthe-et-Moselle 2024-04-12   282.994141
           Deux-Sèvres        2024-04-12   284.964844

[7039096 rows x 1 columns]

In [8]:
mean_temperature_deps_wide = mean_temperature_deps["temperature"].unstack('departement')
mean_temperature_deps_wide

departement                            Ain       Aisne      Allier  \
time       valid_time                                                
2022-02-01 2022-02-01 00:00:00  272.910309  273.894684  273.597595   
           2022-02-01 01:00:00  272.487671  273.567322  272.817169   
           2022-02-01 02:00:00  272.277100  273.376587  272.268005   
           2022-02-01 03:00:00  272.180145  273.396942  271.964752   
           2022-02-01 04:00:00  272.007233  273.682159  271.842896   
...                                    ...         ...         ...   
2024-04-08 2024-04-11 20:00:00  283.846771  285.413727  284.400574   
           2024-04-11 21:00:00  282.891510  284.565247  283.351288   
           2024-04-11 22:00:00  282.144501  283.868805  282.578674   
           2024-04-11 23:00:00  281.615204  283.345581  282.007599   
           2024-04-12 00:00:00  281.138550  282.985352  281.544708   

departement                     Alpes-Maritimes  Alpes-de-Haute-Provence  \
time       valid_time                                                      
2022-02-01 2022-02-01 00:00:00       273.183960               271.443481   
           2022-02-01 01:00:00       272.555573               270.683350   
           2022-02-01 02:00:00       272.001556               270.109314   
           2022-02-01 03:00:00       271.685211               269.690002   
           2022-02-01 04:00:00       271.496704               269.597992   
...                                         ...                      ...   
2024-04-08 2024-04-11 20:00:00       282.918579               282.662598   
           2024-04-11 21:00:00       282.263397               281.692566   
           2024-04-11 22:00:00       281.799591               281.062927   
           2024-04-11 23:00:00       281.460175               280.614441   
           2024-04-12 00:00:00       281.277374               280.328827   

departement                       Ardennes     Ardèche      Ariège  \
time       valid_time                                                
2022-02-01 2022-02-01 00:00:00  274.517609  273.935089  274.208496   
           2022-02-01 01:00:00  274.130096  273.202362  273.834320   
           2022-02-01 02:00:00  273.620361  272.815674  273.667297   
           2022-02-01 03:00:00  273.179688  272.586914  273.482117   
           2022-02-01 04:00:00  273.000122  272.430786  273.347565   
...                                    ...         ...         ...   
2024-04-08 2024-04-11 20:00:00  285.510529  284.053497  281.592010   
           2024-04-11 21:00:00  284.759003  283.390350  280.656799   
           2024-04-11 22:00:00  284.217255  282.944794  280.004059   
           2024-04-11 23:00:00  283.805389  282.679413  279.647522   
           2024-04-12 00:00:00  283.412964  282.385132  279.528656   

departement                           Aube        Aude  ...  Tarn-et-Garonne  \
time       valid_time                                   ...                    
2022-02-01 2022-02-01 00:00:00  274.222931  278.048523  ...       277.390778   
           2022-02-01 01:00:00  274.114258  277.662964  ...       276.848114   
           2022-02-01 02:00:00  273.721069  277.356140  ...       276.190704   
           2022-02-01 03:00:00  273.471863  277.030365  ...       275.635895   
           2022-02-01 04:00:00  273.311310  276.705566  ...       275.253571   
...                                    ...         ...  ...              ...   
2024-04-08 2024-04-11 20:00:00  286.187775  286.689301  ...       285.703766   
           2024-04-11 21:00:00  285.289490  285.805664  ...       284.559845   
           2024-04-11 22:00:00  284.501801  285.129089  ...       283.687988   
           2024-04-11 23:00:00  283.843781  284.572021  ...       283.013550   
           2024-04-12 00:00:00  283.363037  284.161774  ...       282.461609   

departement                     Val-d'Oise  Val-de-Marne         Var  \
time       valid_time                                                  
2022

In [9]:
steps = mean_temperature_deps_wide.index.get_level_values('valid_time') - mean_temperature_deps_wide.index.get_level_values('time')
steps

TimedeltaIndex(['0 days 00:00:00', '0 days 01:00:00', '0 days 02:00:00',
                '0 days 03:00:00', '0 days 04:00:00', '0 days 05:00:00',
                '0 days 06:00:00', '0 days 07:00:00', '0 days 08:00:00',
                '0 days 09:00:00',
                ...
                '3 days 15:00:00', '3 days 16:00:00', '3 days 17:00:00',
                '3 days 18:00:00', '3 days 19:00:00', '3 days 20:00:00',
                '3 days 21:00:00', '3 days 22:00:00', '3 days 23:00:00',
                '4 days 00:00:00'],
               dtype='timedelta64[ns]', length=74884, freq=None)

In [10]:

for i in range(0, 4):
    if i < 3:
        d_index = (steps < pd.Timedelta(days=i+1)) & (steps >= pd.Timedelta(days=i))
    else:
        d_index = (steps >= pd.Timedelta(days=i))
    tmp_selection = mean_temperature_deps_wide.loc[d_index]
    tmp_selection.index = tmp_selection.index.droplevel('time')
    filepath = ROOT_DIR / 'data' / 'silver'/ 'weather_forecasts'  / f'temperature_deps_d{i}.csv'
    tmp_selection.to_csv(filepath)


## Sun Flux

In [11]:
filename_all_sun = ROOT_DIR / 'data' / 'silver' / 'sun_flux_downward.nc'

da_sun = xr.open_dataarray(filename_all_sun,
                            chunks={'time':70}
    )
da_sun

<xarray.DataArray 'sun_flux' (time: 772, step: 96, latitude: 97, longitude: 143)> Size: 4GB
dask.array<open_dataset-sun_flux, shape=(772, 96, 97, 143), dtype=float32, chunksize=(70, 96, 97, 143), chunktype=numpy.ndarray>
Coordinates:
  * step        (step) timedelta64[ns] 768B 01:00:00 ... 4 days 00:00:00
  * latitude    (latitude) float64 776B 51.0 50.9 50.8 50.7 ... 41.6 41.5 41.4
  * longitude   (longitude) float64 1kB -4.7 -4.6 -4.5 -4.4 ... 9.2 9.3 9.4 9.5
  * time        (time) datetime64[ns] 6kB 2022-02-01 2022-02-02 ... 2024-04-08
    valid_time  (time, step) datetime64[ns] 593kB dask.array<chunksize=(70, 96), meta=np.ndarray>
Attributes: (12/29)
    GRIB_paramId:                             169
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      386061
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            accum
    ...                                       ...
    GRIB_name:                                Surface short-wave (solar) radi...
    GRIB_shortName:                           ssrd
    GRIB_units:                               J m**-2
    long_name:                                Surface short-wave (solar) radi...
    units:                                    J m**-2
    standard_name:                            surface_downwelling_shortwave_f...

In [12]:
mean_sunflux_deps = calculate_mean_group_value(
    masks=mask_deps,
    names=departement_names,
    label='departement',
    da_value=da_sun,
    min_lon=france_bounds['min_lon'],
    max_lon=france_bounds['max_lon'],
    min_lat=france_bounds['min_lat'],
    max_lat=france_bounds['max_lat'],
)["sun_flux"].unstack('departement')
mean_sunflux_deps

departement                              Ain         Aisne        Allier  \
time       valid_time                                                      
2022-02-01 2022-02-01 01:00:00  5.379581e-11  5.188652e-11  5.340100e-11   
           2022-02-01 02:00:00  1.076268e-10  1.037731e-10  1.068015e-10   
           2022-02-01 03:00:00  1.614691e-10  1.556609e-10  1.602018e-10   
           2022-02-01 04:00:00  1.928259e-10  1.928258e-10  1.928258e-10   
           2022-02-01 05:00:00  2.410331e-10  2.410331e-10  2.410330e-10   
...                                      ...           ...           ...   
2024-04-08 2024-04-11 20:00:00  6.289667e+07  5.804266e+07  7.158495e+07   
           2024-04-11 21:00:00  6.289667e+07  5.804266e+07  7.158495e+07   
           2024-04-11 22:00:00  6.289667e+07  5.804266e+07  7.158495e+07   
           2024-04-11 23:00:00  6.289667e+07  5.804266e+07  7.158495e+07   
           2024-04-12 00:00:00  6.289667e+07  5.804266e+07  7.158495e+07   

departement                     Alpes-Maritimes  Alpes-de-Haute-Provence  \
time       valid_time                                                      
2022-02-01 2022-02-01 01:00:00     5.667212e-11             5.876055e-11   
           2022-02-01 02:00:00     1.133419e-10             1.175136e-10   
           2022-02-01 03:00:00     1.700074e-10             1.762600e-10   
           2022-02-01 04:00:00     1.928259e-10             1.928259e-10   
           2022-02-01 05:00:00     2.410330e-10             2.410331e-10   
...                                         ...                      ...   
2024-04-08 2024-04-11 20:00:00     6.739630e+07             7.032082e+07   
           2024-04-11 21:00:00     6.739630e+07             7.032082e+07   
           2024-04-11 22:00:00     6.739630e+07             7.032082e+07   
           2024-04-11 23:00:00     6.739630e+07             7.032082e+07   
           2024-04-12 00:00:00     6.739630e+07             7.032082e+07   

departement                         Ardennes       Ardèche        Ariège  \
time       valid_time                                                      
2022-02-01 2022-02-01 01:00:00  5.221113e-11  5.226698e-11  6.675229e-11   
           2022-02-01 02:00:00  1.044223e-10  1.045344e-10  1.338315e-10   
           2022-02-01 03:00:00  1.566333e-10  1.568010e-10  2.011397e-10   
           2022-02-01 04:00:00  1.928259e-10  1.928259e-10  1.928259e-10   
           2022-02-01 05:00:00  2.410330e-10  2.410330e-10  2.410330e-10   
...                                      ...           ...           ...   
2024-04-08 2024-04-11 20:00:00  5.423052e+07  7.739993e+07  7.381190e+07   
           2024-04-11 21:00:00  5.423052e+07  7.739993e+07  7.381190e+07   
           2024-04-11 22:00:00  5.423052e+07  7.739993e+07  7.381190e+07   
           2024-04-11 23:00:00  5.423052e+07  7.739993e+07  7.381190e+07   
           2024-04-12 00:00:00  5.423052e+07  7.739993e+07  7.381190e+07   

departement                             Aube          Aude  ...  \
time       valid_time                                       ...   
2022-02-01 2022-02-01 01:00:00  5.155602e-11  5.281991e-11  ...   
           2022-02-01 02:00:00  1.031121e-10  1.056540e-10  ...   
           2022-02-01 03:00:00  1.546676e-10  1.585065e-10  ...   
           2022-02-01 04:00:00  1.928259e-10  1.928259e-10  ...   
           2022-02-01 05:00:00  2.410330e-10  2.410331e-10  ...   
...                                      ...           ...  ...   
2024-04-08 2024-04-11 20:00:00  6.844169e+07  7.242182e+07  ...   
           2024-04-11 21:00:00  6.844169e+07  7.242182e+07  ...   
           2024-04-11 22:00:00  6.844169e+07  7.242182e+07  ...   
           2024-04-11 23:00:00  6.844169e+07  7.242182e+07  ...   
           2024-04-12 00:00:00  6.844169e+07  7.242182e+07  ...   

departement                     Tarn-et-Garonne    Val-d'Oise  Val-de-Marne  \
time       valid_time                                                         
202

In [13]:
mean_sunflux_deps_instant = instant_flux_from_cumul(mean_sunflux_deps)
mean_sunflux_deps_instant

departement                              Ain         Aisne        Allier  \
time       valid_time                                                      
2022-02-01 2022-02-01 00:00:00  5.379581e-11  5.188652e-11  5.340100e-11   
           2022-02-01 01:00:00  5.383094e-11  5.188655e-11  5.340053e-11   
           2022-02-01 02:00:00  5.384233e-11  5.188787e-11  5.340024e-11   
           2022-02-01 03:00:00  3.135678e-11  3.716485e-11  3.262407e-11   
           2022-02-01 04:00:00  4.820719e-11  4.820726e-11  4.820716e-11   
...                                      ...           ...           ...   
2024-04-08 2024-04-11 19:00:00  0.000000e+00  0.000000e+00  0.000000e+00   
           2024-04-11 20:00:00  0.000000e+00  0.000000e+00  0.000000e+00   
           2024-04-11 21:00:00  0.000000e+00  0.000000e+00  0.000000e+00   
           2024-04-11 22:00:00  0.000000e+00  0.000000e+00  0.000000e+00   
           2024-04-11 23:00:00  0.000000e+00  0.000000e+00  0.000000e+00   

departement                     Alpes-Maritimes  Alpes-de-Haute-Provence  \
time       valid_time                                                      
2022-02-01 2022-02-01 00:00:00     5.667212e-11             5.876055e-11   
           2022-02-01 01:00:00     5.666977e-11             5.875309e-11   
           2022-02-01 02:00:00     5.666549e-11             5.874641e-11   
           2022-02-01 03:00:00     2.281847e-11             1.656583e-11   
           2022-02-01 04:00:00     4.820715e-11             4.820717e-11   
...                                         ...                      ...   
2024-04-08 2024-04-11 19:00:00     0.000000e+00             0.000000e+00   
           2024-04-11 20:00:00     0.000000e+00             0.000000e+00   
           2024-04-11 21:00:00     0.000000e+00             0.000000e+00   
           2024-04-11 22:00:00     0.000000e+00             0.000000e+00   
           2024-04-11 23:00:00     0.000000e+00             0.000000e+00   

departement                         Ardennes       Ardèche        Ariège  \
time       valid_time                                                      
2022-02-01 2022-02-01 00:00:00  5.221113e-11  5.226698e-11  6.675229e-11   
           2022-02-01 01:00:00  5.221116e-11  5.226745e-11  6.707917e-11   
           2022-02-01 02:00:00  5.221105e-11  5.226657e-11  6.730826e-11   
           2022-02-01 03:00:00  3.619251e-11  3.602486e-11 -8.313836e-12   
           2022-02-01 04:00:00  4.820715e-11  4.820715e-11  4.820712e-11   
...                                      ...           ...           ...   
2024-04-08 2024-04-11 19:00:00  0.000000e+00  0.000000e+00  0.000000e+00   
           2024-04-11 20:00:00  0.000000e+00  0.000000e+00  0.000000e+00   
           2024-04-11 21:00:00  0.000000e+00  0.000000e+00  0.000000e+00   
           2024-04-11 22:00:00  0.000000e+00  0.000000e+00  0.000000e+00   
           2024-04-11 23:00:00  0.000000e+00  0.000000e+00  0.000000e+00   

departement                             Aube          Aude  ...  \
time       valid_time                                       ...   
2022-02-01 2022-02-01 00:00:00  5.155602e-11  5.281991e-11  ...   
           2022-02-01 01:00:00  5.155604e-11  5.283414e-11  ...   
           2022-02-01 02:00:00  5.155558e-11  5.285241e-11  ...   
           2022-02-01 03:00:00  3.815821e-11  3.431942e-11  ...   
           2022-02-01 04:00:00  4.820715e-11  4.820720e-11  ...   
...                                      ...           ...  ...   
2024-04-08 2024-04-11 19:00:00  0.000000e+00  0.000000e+00  ...   
           2024-04-11 20:00:00  0.000000e+00  0.000000e+00  ...   
           2024-04-11 21:00:00  0.000000e+00  0.000000e+00  ...   
           2024-04-11 22:00:00  0.000000e+00  0.000000e+00  ...   
           2024-04-11 23:00:00  0.000000e+00  0.000000e+00  ...   

departement                     Tarn-et-Garonne    Val-d'Oise  Val-de-Marne  \
time       valid_time                                                         
202

In [14]:
steps = mean_sunflux_deps_instant.index.get_level_values('valid_time') - mean_sunflux_deps_instant.index.get_level_values('time')

for i in range(0, 4):
    if i < 3:
        d_index = (steps < pd.Timedelta(days=i+1)) & (steps >= pd.Timedelta(days=i))
    else:
        d_index = (steps >= pd.Timedelta(days=i))
    tmp_selection = mean_sunflux_deps_instant.loc[d_index]
    tmp_selection.index = tmp_selection.index.droplevel('time')
    filepath = ROOT_DIR / 'data' / 'silver'/ 'weather_forecasts'  / f'sun_flux_downward_deps_d{i}.csv'
    tmp_selection.to_csv(filepath)

## Wind Speed

In [6]:
da_wind = xr.open_dataarray(ROOT_DIR / 'data' / 'silver' / 'wind_speed.nc',
                            chunks={'time':70}
    )


In [7]:
da_wind

<xarray.DataArray 'wind_speed' (time: 772, step: 97, latitude: 97,
                                longitude: 143)> Size: 4GB
dask.array<open_dataset-wind_speed, shape=(772, 97, 97, 143), dtype=float32, chunksize=(70, 97, 97, 143), chunktype=numpy.ndarray>
Coordinates:
  * step        (step) timedelta64[ns] 776B 00:00:00 ... 4 days 00:00:00
  * latitude    (latitude) float64 776B 51.0 50.9 50.8 50.7 ... 41.6 41.5 41.4
  * longitude   (longitude) float64 1kB -4.7 -4.6 -4.5 -4.4 ... 9.2 9.3 9.4 9.5
  * time        (time) datetime64[ns] 6kB 2022-02-01 2022-02-02 ... 2024-04-08
    valid_time  (time, step) datetime64[ns] 599kB dask.array<chunksize=(70, 97), meta=np.ndarray>
Attributes: (12/29)
    GRIB_paramId:                             207
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      386061
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                10 metre wind speed
    GRIB_shortName:                           10si
    GRIB_units:                               m s**-1
    long_name:                                10 metre wind speed
    units:                                    m s**-1
    standard_name:                            unknown

In [8]:
mean_wind_deps = calculate_mean_group_value(
    masks=mask_deps,
    names=departement_names,
    label='departement',
    da_value=da_wind,
    min_lon=france_bounds['min_lon'],
    max_lon=france_bounds['max_lon'],
    min_lat=france_bounds['min_lat'],
    max_lat=france_bounds['max_lat'],
)["wind_speed"].unstack('departement')

steps = mean_wind_deps.index.get_level_values('valid_time') - mean_wind_deps.index.get_level_values('time')

for i in range(0, 4):
    if i < 3:
        d_index = (steps < pd.Timedelta(days=i+1)) & (steps >= pd.Timedelta(days=i))
    else:
        d_index = (steps >= pd.Timedelta(days=i))
    tmp_selection = mean_wind_deps.loc[d_index]
    tmp_selection.index = tmp_selection.index.droplevel('time')
    filepath = ROOT_DIR / 'data' / 'silver' / 'weather_forecasts' / f'wind_speed_deps_d{i}.csv'
    tmp_selection.to_csv(filepath)

# Region extraction

In [17]:
mask_regions = get_mask_regions()

## Sun Flux

In [18]:
mean_sunflux_regs = calculate_mean_group_value(
    masks=mask_regions,
    names=region_names,
    label='region',
    da_value=da_sun,
    min_lon=france_bounds['min_lon'],
    max_lon=france_bounds['max_lon'],
    min_lat=france_bounds['min_lat'],
    max_lat=france_bounds['max_lat'],
)["sun_flux"].unstack('region')
mean_sunflux_regs_instant = instant_flux_from_cumul(mean_sunflux_regs)

steps = mean_sunflux_regs_instant.index.get_level_values('valid_time') - mean_sunflux_regs_instant.index.get_level_values('time')

for i in range(0, 4):
    if i < 3:
        d_index = (steps < pd.Timedelta(days=i+1)) & (steps >= pd.Timedelta(days=i))
    else:
        d_index = (steps >= pd.Timedelta(days=i))
    tmp_selection = mean_sunflux_regs_instant.loc[d_index]
    tmp_selection.index = tmp_selection.index.droplevel('time')
    filepath = ROOT_DIR / 'data' / 'silver'/ 'weather_forecasts'  / f'sun_flux_downward_regs_d{i}.csv'
    tmp_selection.to_csv(filepath)


## Temperature

In [19]:
mean_temperature_regs = calculate_mean_group_value(
    masks=mask_regions,
    names=region_names,
    label='region',
    da_value=temperature_da,
    min_lon=france_bounds['min_lon'],
    max_lon=france_bounds['max_lon'],
    min_lat=france_bounds['min_lat'],
    max_lat=france_bounds['max_lat'],
)["temperature"].unstack('region')

steps = mean_temperature_regs.index.get_level_values('valid_time') - mean_temperature_regs.index.get_level_values('time')

for i in range(0, 4):
    if i < 3:
        d_index = (steps < pd.Timedelta(days=i+1)) & (steps >= pd.Timedelta(days=i))
    else:
        d_index = (steps >= pd.Timedelta(days=i))
    tmp_selection = mean_temperature_regs.loc[d_index]
    tmp_selection.index = tmp_selection.index.droplevel('time')
    filepath = ROOT_DIR / 'data' / 'silver'/ 'weather_forecasts'  / f'temperature_regs_d{i}.csv'
    tmp_selection.to_csv(filepath)

## Wind Speed

In [20]:
mean_windspeed_regs = calculate_mean_group_value(
    masks=mask_regions,
    names=region_names,
    label='region',
    da_value=da_wind,
    min_lon=france_bounds['min_lon'],
    max_lon=france_bounds['max_lon'],
    min_lat=france_bounds['min_lat'],
    max_lat=france_bounds['max_lat'],
)["wind_speed"].unstack('region')

steps = mean_windspeed_regs.index.get_level_values('valid_time') - mean_windspeed_regs.index.get_level_values('time')

for i in range(0, 4):
    if i < 3:
        d_index = (steps < pd.Timedelta(days=i+1)) & (steps >= pd.Timedelta(days=i))
    else:
        d_index = (steps >= pd.Timedelta(days=i))
    tmp_selection = mean_windspeed_regs.loc[d_index]
    tmp_selection.index = tmp_selection.index.droplevel('time')
    filepath = ROOT_DIR / 'data' / 'silver'/ 'weather_forecasts'  / f'wind_speed_regs_d{i}.csv'
    tmp_selection.to_csv(filepath)